<a href="https://colab.research.google.com/github/UrmilaJasiwar/Face_Mask_Detection/blob/master/Face_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
tensorflow version is  2.3.0
import keras version is 2.4.3
'''

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import ModelCheckpoint

In [ ]:
DIRECTORY = r"/content/drive/My Drive/dataset" # Data Location
CATEGORIES = ["with_mask", "without_mask"]

In [ ]:
#process data
images = []
labels = []
cont = 1
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
      cont += 1
      img_path = os.path.join(path, img)
      image = load_img(img_path, target_size=(299, 299))
      image = img_to_array(image)
      image = preprocess_input(image)
      images.append(image)
      labels.append(category)
      print("impage processed : ",cont)
     

In [ ]:
#performing one_hot_encoding on labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

images = np.array(images, dtype="float32")
labels = np.array(labels)
#split data into train and test
(trainX, testX, trainY, testY) = train_test_split(images, labels,test_size=0.20, stratify=labels, random_state=32)

In [ ]:
#image Augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
#Model
baseModel = InceptionV3(include_top=False,
	input_tensor=Input(shape=(299, 299, 3)))
Model1 = baseModel.output
Model1 = AveragePooling2D(pool_size=(8, 8))(Model1)
Model1 = Flatten(name="flatten")(Model1)
Model1 = Dense(512, activation="relu")(Model1)
Model1 = Dropout(0.5)(Model1)
Model1= Dense(2, activation="softmax")(Model1)
model = Model(inputs=baseModel.input, outputs=Model1)

In [ ]:
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
LR = 1e-4
EPOCHS = 20
BS = 32
opt = Adam(lr=LR, decay=LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [ ]:

filepath = '/content/drive/My Drive/Face_model/model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:

model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS,callbacks=[checkpoint])

Epoch 1/20
95/95 [==============================] - ETA: 0s - loss: 0.1647 - accuracy: 0.9341
Epoch 00001: val_loss improved from inf to 0.03753, saving model to /content/drive/My Drive/Face_model/model-ep001-loss0.165-val_loss0.038.h5
95/95 [==============================] - 59s 619ms/step - loss: 0.1647 - accuracy: 0.9341 - val_loss: 0.0375 - val_accuracy: 0.9896
Epoch 2/20
95/95 [==============================] - ETA: 0s - loss: 0.0478 - accuracy: 0.9832
Epoch 00002: val_loss improved from 0.03753 to 0.02893, saving model to /content/drive/My Drive/Face_model/model-ep002-loss0.048-val_loss0.029.h5
95/95 [==============================] - 56s 588ms/step - loss: 0.0478 - accuracy: 0.9832 - val_loss: 0.0289 - val_accuracy: 0.9935
Epoch 3/20
95/95 [==============================] - ETA: 0s - loss: 0.0396 - accuracy: 0.9885
Epoch 00003: val_loss did not improve from 0.02893
95/95 [==============================] - 54s 573ms/step - loss: 0.0396 - accuracy: 0.9885 - val_loss: 0.0292 - val_

In [ ]:
predIdxs = model.predict(testX, batch_size=BS)

In [ ]:
#Model accuracy report
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       1.00      0.99      0.99       383
without_mask       0.99      1.00      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767



In [ ]:
'''
Change kernel size and Activation Function of headModel parameters, 
feeding More data to the Model, Adding Noise to the Data, 
Alter image  size,Alter color channels would help in adding little accuracy 
to the Model
'''
